## GPU

### GPU info

In [1]:
!nvidia-smi

Fri Feb 23 10:50:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Architecture

### Lib

In [ ]:

# Util #
import pickle
import numpy as np
import os
import scipy.sparse as sp
import torch
from scipy.sparse import linalg
from torch.autograd import Variable

from sklearn.preprocessing import MinMaxScaler

# Layer #
from __future__ import division
import torch
import torch.nn as nn
from torch.nn import init
import numbers
import torch.nn.functional as F

# Model #
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import sys

# Trainer #
import torch.optim as optim
import math

# Main #
import torch
import numpy as np
import argparse
import time
import matplotlib.pyplot as plt


from torch.utils.tensorboard import SummaryWriter

### Util

In [ ]:

class DataLoaderM(object):
    def __init__(self, xs, ys, batch_size, pad_with_last_sample=True):
        """
        :param xs:
        :param ys:
        :param batch_size:
        :param pad_with_last_sample: pad with the last sample to make number of samples divisible to batch_size.
        """
        self.batch_size = batch_size
        self.current_ind = 0

        # 將資料長度補齊至batch_size可整除之數量
        # 補齊方法: 取原資料最後一個並複製多個來補齊
        if pad_with_last_sample:
            # 計算需補齊數量
            num_padding = (batch_size - (len(xs) % batch_size)) % batch_size
            x_padding = np.repeat(xs[-1:], num_padding, axis=0)
            y_padding = np.repeat(ys[-1:], num_padding, axis=0)

            # 將複製後的ele進行concatenate以補齊成可整除batch_size之長度
            xs = np.concatenate([xs, x_padding], axis=0)
            ys = np.concatenate([ys, y_padding], axis=0)
        self.size = len(xs)
        self.num_batch = int(self.size // self.batch_size)
        self.xs = xs
        self.ys = ys

    #**fusion**#
    def set_permutation(self, permutation):
        assert len(permutation) == self.size, "Permutation length must match data size"
        self.xs = self.xs[permutation]
        self.ys = self.ys[permutation]
    '''
    def shuffle(self):
        permutation = np.random.permutation(self.size)
        xs, ys = self.xs[permutation], self.ys[permutation]
        self.xs = xs
        self.ys = ys
    '''

    def get_iterator(self):
        self.current_ind = 0
        def _wrapper():
            while self.current_ind < self.num_batch:
                start_ind = self.batch_size * self.current_ind
                end_ind = min(self.size, self.batch_size * (self.current_ind + 1))
                x_i = self.xs[start_ind: end_ind, ...]
                y_i = self.ys[start_ind: end_ind, ...]
                # 節省記憶體:
                # yield 設計來的目的，就是為了單次輸出內容
                # 我們可以把 yield 暫時看成 return，但是這個 return 的功能只有單次
                # 而且，一旦我們的程式執行到 yield 後，程式就會把值丟出，並暫時停止
                yield (x_i, y_i)
                self.current_ind += 1

        return _wrapper()

class StandardScaler():
    """
    Standard the input
    """
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std
    def transform(self, data):
        return (data - self.mean) / self.std
    def inverse_transform(self, data):
        return (data * self.std) + self.mean

def asym_adj(adj):
    """Asymmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1)).flatten()
    d_inv = np.power(rowsum, -1).flatten()
    d_inv[np.isinf(d_inv)] = 0.
    d_mat= sp.diags(d_inv)
    return d_mat.dot(adj).astype(np.float32).todense()


def load_pickle(pickle_file):
    try:
        with open(pickle_file, 'rb') as f:
            pickle_data = pickle.load(f)
    except UnicodeDecodeError as e:
        with open(pickle_file, 'rb') as f:
            pickle_data = pickle.load(f, encoding='latin1')
    except Exception as e:
        print('Unable to load data ', pickle_file, ':', e)
        raise
    return pickle_data

def load_adj(pkl_filename, adjtype):
    sensor_ids, sensor_id_to_ind, adj_mx = load_pickle(pkl_filename)

    print('# 全部L.A.的sensor ID(sensor_ids):\n',sensor_ids)
    print('# 將sensor ID對應index(sensor_id_to_ind):\n',sensor_id_to_ind)

    if adjtype == "scalap":
        adj = [calculate_scaled_laplacian(adj_mx)]
    elif adjtype == "normlap":
        adj = [calculate_normalized_laplacian(adj_mx).astype(np.float32).todense()]
    elif adjtype == "symnadj":
        adj = [sym_adj(adj_mx)]
    elif adjtype == "transition":
        adj = [asym_adj(adj_mx)]
    elif adjtype == "doubletransition":
        adj = [asym_adj(adj_mx), asym_adj(np.transpose(adj_mx))]   # asym_adj(adj_mx): forward transition matrix / asym_adj(np.transpose(adj_mx)): backward transition matrix
    elif adjtype == "identity":
        adj = [np.diag(np.ones(adj_mx.shape[0])).astype(np.float32)]
    else:
        error = 0
        assert error, "adj type not defined"

    print('# Double transition Transition matrix of Eq 4:\n',adj)
    return sensor_ids, sensor_id_to_ind, adj


def masked_mse(preds, labels, null_val=np.nan):
    if np.isnan(null_val):
        mask = ~torch.isnan(labels)
    else:
        mask = (labels!=null_val)
    mask = mask.float()
    mask /= torch.mean((mask))
    mask = torch.where(torch.isnan(mask), torch.zeros_like(mask), mask)
    loss = (preds-labels)**2
    loss = loss * mask
    loss = torch.where(torch.isnan(loss), torch.zeros_like(loss), loss)
    return torch.mean(loss)

def masked_rmse(preds, labels, null_val=np.nan):
    return torch.sqrt(masked_mse(preds=preds, labels=labels, null_val=null_val))


def masked_mae(preds, labels, null_val=np.nan):
    if np.isnan(null_val):
        mask = ~torch.isnan(labels)
    else:
        mask = (labels!=null_val)
    mask = mask.float()
    mask /=  torch.mean((mask))
    mask = torch.where(torch.isnan(mask), torch.zeros_like(mask), mask)
    loss = torch.abs(preds-labels)
    loss = loss * mask
    loss = torch.where(torch.isnan(loss), torch.zeros_like(loss), loss)
    return torch.mean(loss)
def masked_mape(preds, labels, null_val=np.nan):
    if np.isnan(null_val):
        mask = ~torch.isnan(labels)
    else:
        mask = (labels!=null_val)
    mask = mask.float()
    mask /=  torch.mean((mask))
    mask = torch.where(torch.isnan(mask), torch.zeros_like(mask), mask)
    loss = torch.abs(preds-labels)/labels
    #loss = 2.0 * torch.mean(torch.abs(preds - labels) / (torch.abs(preds) + torch.abs(labels)))
    loss = loss * mask
    loss = torch.where(torch.isnan(loss), torch.zeros_like(loss), loss)
    return torch.mean(loss)
def masked_smape(preds, labels, null_val=np.nan):
    if np.isnan(null_val):
        mask = ~torch.isnan(labels)
    else:
        mask = (labels!=null_val)
    mask = mask.float()
    mask /=  torch.mean((mask))
    mask = torch.where(torch.isnan(mask), torch.zeros_like(mask), mask)
    #loss = torch.abs(preds-labels)/labels
    loss = 2.0 * (torch.abs(preds - labels) / (torch.abs(preds) + torch.abs(labels)))
    loss = loss * mask
    loss = torch.where(torch.isnan(loss), torch.zeros_like(loss), loss)
    return torch.mean(loss)

def metric(pred, real):
    mae = masked_mae(pred,real,0.0).item()
    mape = masked_mape(pred,real,0.0).item()
    rmse = masked_rmse(pred,real,0.0).item()
    smape = masked_smape(pred,real,0.0).item()
    return mae,mape,rmse,smape

### Layer

In [ ]:


class linear(nn.Module):
    def __init__(self,c_in,c_out,bias=True):
        super(linear,self).__init__()
        self.mlp = torch.nn.Conv2d(c_in, c_out, kernel_size=(1, 1), padding=(0,0), stride=(1,1), bias=bias)

    def forward(self,x):
        return self.mlp(x)

class LayerNorm(nn.Module):
    __constants__ = ['normalized_shape', 'weight', 'bias', 'eps', 'elementwise_affine']
    def __init__(self, normalized_shape, eps=1e-5, elementwise_affine=True):
        super(LayerNorm, self).__init__()
        if isinstance(normalized_shape, numbers.Integral):
            normalized_shape = (normalized_shape,)
        self.normalized_shape = tuple(normalized_shape)
        self.eps = eps
        self.elementwise_affine = elementwise_affine
        if self.elementwise_affine:
            self.weight = nn.Parameter(torch.Tensor(*normalized_shape))
            self.bias = nn.Parameter(torch.Tensor(*normalized_shape))
        else:
            self.register_parameter('weight', None)
            self.register_parameter('bias', None)
        self.reset_parameters()


    def reset_parameters(self):
        if self.elementwise_affine:
            init.ones_(self.weight)
            init.zeros_(self.bias)

    def forward(self, input, idx):
        if self.elementwise_affine:
            return F.layer_norm(input, tuple(input.shape[1:]), self.weight[:,idx,:], self.bias[:,idx,:], self.eps)
        else:
            return F.layer_norm(input, tuple(input.shape[1:]), self.weight, self.bias, self.eps)

    def extra_repr(self):
        return '{normalized_shape}, eps={eps}, ' \
            'elementwise_affine={elementwise_affine}'.format(**self.__dict__)

### MGAT (for Enhancement step)

In [ ]:

class A_GMAT_base(nn.Module):
    def __init__(self, n_heads, in_channel, num_nodes, dropout, bias=True):
        super(A_GMAT_base, self).__init__()

        print('A_GMAT_base', n_heads, in_channel, num_nodes, dropout)
        self.n_head = n_heads
        self.f_in = num_nodes
        self.a_src = nn.Parameter(torch.Tensor(self.n_head, 10, 1))
        self.a_dst = nn.Parameter(torch.Tensor(self.n_head, 10, 1))

        self.leaky_relu = nn.LeakyReLU(negative_slope=0.2)
        self.softmax = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)
        if bias:
            self.bias = nn.Parameter(torch.Tensor(10))
            nn.init.constant_(self.bias, 0)
        else:
            self.register_parameter("bias", None)

        nn.init.xavier_uniform_(self.a_src, gain=1.414)
        nn.init.xavier_uniform_(self.a_dst, gain=1.414)

    def forward(self, h):
        bs, ch, n, dim = h.size()
        h_prime = h
        attn_src = torch.matmul(h, self.a_src)
        attn_dst = torch.matmul(h, self.a_dst)
        attn = attn_src.expand(-1, -1, -1, n) + attn_dst.expand(-1, -1, -1, n).permute(
            0, 1, 3, 2
        )
        attn = self.leaky_relu(attn)
        attn = self.softmax(attn)
        attn = self.dropout(attn)
        output = torch.matmul(attn, h_prime)
        return output + self.bias, attn

class A_GMAT(nn.Module):
    def __init__(self, n_heads, in_channel, num_nodes, dropout, alpha):
        super(A_GMAT, self).__init__()

        self.dropout = dropout

        self.layer = A_GMAT_base(
                    n_heads, in_channel, num_nodes, dropout
                )

    def forward(self, x):
        bs,ch,n,dim = x.size()
        x, attn = self.layer(x)

        return x


class A_GMAT_module(nn.Module):
    def __init__(self, n_heads, in_channel, num_nodes, mlp, mlp2, dropout, alpha):
        super(A_GMAT_module, self).__init__()
        print('A_GMAT_module', n_heads, in_channel, num_nodes, dropout, alpha)
        self.net = A_GMAT(n_heads, in_channel, num_nodes, dropout, alpha)

        self.mlp_convs = nn.ModuleList()
        self.mlp_bns = nn.ModuleList()
        last_channel = 32
        for out_channel in mlp:
            self.mlp_convs.append(nn.Conv2d(last_channel, out_channel, 1))
            last_channel = out_channel

        self.mlp_convs2 = nn.ModuleList()
        self.mlp_bns2 = nn.ModuleList()
        last_channel = n_heads
        for out_channel in mlp2:
            self.mlp_convs2.append(nn.Conv2d(last_channel, out_channel, 1))
            last_channel = out_channel

        self.dropout1 = nn.Dropout(0.3)
        self.dropout2 = nn.Dropout(0.6)
    def forward(self,x):
        #print('in MGAT x.shape',x.shape)
        #sys.exit()
        bs, ch, n, dim = x.size()

        x_input = x

        x_all = []
        x_input_cpy = x_input

        for i, conv in enumerate(self.mlp_convs):
          x_input = ((conv(x_input)))

        x_input_cpy2 = x_input
        x_input = self.net(x_input)
        x_input = x_input_cpy2+ self.dropout1(x_input)

        for i, conv in enumerate(self.mlp_convs2):
          x_input = F.relu((conv(x_input)))

        x_input = x_input_cpy+ self.dropout2(x_input)

        return x_input


### STGNN's architecture

In [ ]:

# this efficient implementation comes from https://github.com/xptree/DeepInf/
class BatchMultiHeadGraphAttention(nn.Module):
    def __init__(self, n_heads, in_channel, num_nodes, dropout, bias=True):
        super(BatchMultiHeadGraphAttention, self).__init__()

        print('BatchMultiHeadGraphAttention', n_heads, in_channel, num_nodes, dropout)
        self.n_head = n_heads
        self.f_in = num_nodes
        #self.w = nn.Parameter(torch.Tensor(self.n_head, num_nodes, num_nodes))
        self.a_src = nn.Parameter(torch.Tensor(self.n_head, num_nodes, 1))
        self.a_dst = nn.Parameter(torch.Tensor(self.n_head, num_nodes, 1))

        self.leaky_relu = nn.LeakyReLU(negative_slope=0.2)
        self.softmax = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)
        if bias:
            self.bias = nn.Parameter(torch.Tensor(num_nodes))
            nn.init.constant_(self.bias, 0)
        else:
            self.register_parameter("bias", None)

        #nn.init.xavier_uniform_(self.w, gain=1.414)
        nn.init.xavier_uniform_(self.a_src, gain=1.414)
        nn.init.xavier_uniform_(self.a_dst, gain=1.414)

    def forward(self, h):
        bs, ch, n, dim = h.size()
        #h_prime = torch.matmul(h, self.w)
        h_prime = h
        attn_src = torch.matmul(h, self.a_src)
        attn_dst = torch.matmul(h, self.a_dst)
        attn = attn_src.expand(-1, -1, -1, n) + attn_dst.expand(-1, -1, -1, n).permute(
            0, 1, 3, 2
        )

        '''
        ##############
        '''

        attn = self.leaky_relu(attn)
        attn = self.softmax(attn)
        attn = self.dropout(attn)
        output = torch.matmul(attn, h_prime)
        return output + self.bias, attn

class GAT(nn.Module):
    def __init__(self, n_heads, in_channel, num_nodes, dropout, alpha):
        super(GAT, self).__init__()

        self.dropout = dropout

        self.gat_layer = BatchMultiHeadGraphAttention(
                    n_heads, in_channel, num_nodes, dropout
                )

        #self.norm = torch.nn.InstanceNorm2d(32).cuda()

    def forward(self, x):
        bs,ch,n,dim = x.size()
        #x = self.norm(x) # instance norm for 32 channel
        x, attn = self.gat_layer(x)
        #x = F.elu(x.transpose(1, 2).contiguous().view(bs, ch, n, -1))

        return x


class GATEncoder(nn.Module):
    def __init__(self, kern, dilation_factor, temporal_len, n_heads, in_channel, num_nodes, mlp, mlp2, dropout, alpha):
        super(GATEncoder, self).__init__()

        print('GATEncoder', n_heads, in_channel, num_nodes, dropout, alpha)
        self.gat_net = GAT(n_heads, in_channel, num_nodes, dropout, alpha)

        self.mlp_convs = nn.ModuleList()
        self.mlp_bns = nn.ModuleList()
        last_channel = 32
        for out_channel in mlp:
            self.mlp_convs.append(nn.Conv2d(last_channel, out_channel, 1))
            last_channel = out_channel

        self.mlp_convs2 = nn.ModuleList()
        self.mlp_bns2 = nn.ModuleList()
        last_channel = n_heads
        print('mlp2', mlp2)
        for out_channel in mlp2:
            self.mlp_convs2.append(nn.Conv2d(last_channel, out_channel, 1))
            last_channel = out_channel

        #self.lay_norm = nn.LayerNorm([32, temporal_len, num_nodes])
        #self.lay_norm2 = nn.LayerNorm([n_heads,temporal_len, num_nodes])

        #self.bn_norm1 = nn.BatchNorm2d(8)
        self.bn_norm2 = nn.BatchNorm2d(out_channel)
        self.bn_norm3 = nn.BatchNorm2d(out_channel)

        self.dropout1 = nn.Dropout(0.3)
        self.dropout2 = nn.Dropout(0.2)

        self.mlp = (nn.Conv2d(32,32,(1,kern),dilation=(1,dilation_factor)))

    def forward(self,x):

        bs, ch, n, dim = x.size()

        x_input = x.permute(0,1,3,2)
        x_input_cpy = x_input

        #-------------relu(CNN)-------------#
        for i, conv in enumerate(self.mlp_convs):
            #print('1x_input in', x_input.shape)
            x_input = F.relu((conv(x_input)))
            #print('1x_input out', x_input.shape)
        #-------------relu(CNN)-------------#

        #-------------GAT-------------#
        x_input_cpy2 = x_input

        x_input = self.gat_net(x_input)

        x_input = x_input_cpy2+ self.dropout1(x_input)
        #-------------GAT-------------#

        #print('x_input1', x_input.shape)
        #-------------relu(CNN)-------------#
        for i, conv in enumerate(self.mlp_convs2):
          #print('x_input in', x_input.shape)
          x_input = F.relu((conv(x_input)))
          #print('x_input out', x_input.shape)
        #-------------relu(CNN)-------------#
        #print('x_input', x_input.shape)
        x_input = (x_input_cpy + self.dropout2(x_input)).permute(0,1,3,2)

        x_input = self.bn_norm2(x_input)

        #最後一維度緊收
        x_input = F.relu(self.mlp(x_input))
        x_input = self.bn_norm3(x_input)

        return x_input

import numpy as np

def create_matrix(n, k):
    mat = np.zeros((n, n))
    for i in range(n):
        for j in range(k):
            if i-j >= 0:
                mat[i][i-j] = 1
    return mat


def pearson_corr2(tensor): # all: (64,1,n,dim)
  # Input tensor shape: (batch_size, num_nodes, time_steps)
  batch_size, num_nodes, _ = tensor.shape
  tensor = tensor - tensor.mean(dim=2, keepdim=True)
  std = tensor.std(dim=2, keepdim=True)
  tensor = tensor / (std + 1e-8)
  correlation_matrix = torch.matmul(tensor, tensor.transpose(1, 2))
  correlation_matrix = correlation_matrix / (tensor.shape[2] - 1)
  return correlation_matrix


def topK(attn, top_num ):

  # Get the top K values and their indices for each row
  top_k_values, top_k_indices = attn.topk(top_num, dim=3)

  # Create a mask with the same shape as the input tensor, filled with zeros
  mask = torch.zeros_like(attn)

  # Set the top K values in the mask to 1
  mask.scatter_(3, top_k_indices, 1)

  # Multiply the input tensor with the mask to get the result
  attn = attn * mask

  return  attn



# this efficient implementation comes from https://github.com/xptree/DeepInf/
class S_BatchMultiHeadGraphAttention(nn.Module):
    def __init__(self, n_heads, num_nodes, dropout, bias=True):
        super(S_BatchMultiHeadGraphAttention, self).__init__()

        print('S_BatchMultiHeadGraphAttention', n_heads, num_nodes, dropout)
        self.n_head = n_heads
        self.f_in = num_nodes
        #self.w = nn.Parameter(torch.Tensor(self.n_head, num_nodes, num_nodes))
        self.a_src = nn.Parameter(torch.Tensor(self.n_head*2, num_nodes, 1))
        self.a_dst = nn.Parameter(torch.Tensor(self.n_head*2, num_nodes, 1))

        self.leaky_relu = nn.LeakyReLU(negative_slope=0.2)
        self.softmax = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)
        if bias:
            self.bias = nn.Parameter(torch.Tensor(num_nodes))
            nn.init.constant_(self.bias, 0)
        else:
            self.register_parameter("bias", None)


        nn.init.xavier_uniform_(self.a_src, gain=1.414)
        nn.init.xavier_uniform_(self.a_dst, gain=1.414)

        self.W_si = nn.Parameter(torch.zeros(size=(1, 1)))
        nn.init.xavier_uniform_(self.W_si.data, gain=1.414)
        self.W_ei = nn.Parameter(torch.zeros(size=(1, 1)))
        nn.init.xavier_uniform_(self.W_ei.data, gain=1.414)

    def forward(self, h, a):

        bs, ch, n, dim = h.size()
        #h_prime = torch.matmul(h, self.w)
        h_prime = h
        attn_src = torch.matmul(torch.tanh(h), self.a_src)
        attn_dst = torch.matmul(torch.tanh(h), self.a_dst)
        attn = attn_src.expand(-1, -1, -1, n) + attn_dst.expand(-1, -1, -1, n).permute(
            0, 1, 3, 2
        )

        attn_2 = self.leaky_relu(attn)
        attn_2 = self.softmax(attn_2)
        attn_2 = self.dropout(attn_2)

        attn_2 = abs(self.W_ei)*attn_2+abs(self.W_si)*a

        output_2 = torch.matmul(attn_2, h_prime)

        return output_2, attn

# MutiChannel_GAT(kern, dilation_factor, n_heads, num_nodes, mlp, mlp2, dropout)
class S_MutiChannel_GAT(nn.Module):
    def __init__(self, kern, dilation_factor, n_heads, num_nodes, mlp, mlp2, dropout):
        super(S_MutiChannel_GAT, self).__init__()

        print('S_MutiChannel_GAT', n_heads, num_nodes, dropout)

        self.gat_layer = S_BatchMultiHeadGraphAttention(
            n_heads, num_nodes, dropout
        )
        self.gat_layer2 = S_BatchMultiHeadGraphAttention(
            n_heads, num_nodes, dropout
        )

        self.mlp1 =  nn.Conv2d(in_channels=32,
                                    out_channels=16,
                                    kernel_size=(1, 1))

        self.mlp2 =  nn.Conv2d(in_channels=48,
                                    out_channels=32,
                                    kernel_size=(1, 1))

    def forward(self,x_input, a_f):
        x_input_cpy = x_input

        x = x_input
        x = self.mlp1(x)

        x1, attn = self.gat_layer(x,a_f)
        a_f = torch.matmul(a_f,a_f)
        x2, attn = self.gat_layer2(x1,a_f)

        x_out = self.mlp2(torch.cat([x,x1,x2],dim=1))

        return x_out

### Framework

In [ ]:


class gginet(nn.Module):
    def __init__(self, model_type,  num_nodes, device, predefined_A=None,kernel_set=None, static_feat=None, dropout=0.3, node_dim=40, dilation_exponential=1, conv_channels=32, residual_channels=32, skip_channels=64, end_channels=128, seq_length=12, in_dim=2, out_dim=12, layers=3, propalpha=0.05, tanhalpha=3, layer_norm_affline=True):
        super(gginet, self).__init__()

        self.model_type = model_type

        self.num_nodes = num_nodes
        self.dropout = dropout
        self.predefined_A = predefined_A
        self.layers = layers
        self.seq_length = seq_length

        self.t_gat1 = nn.ModuleList()
        self.t_gat2 = nn.ModuleList()

        self.residual_convs = nn.ModuleList()
        self.skip_convs = nn.ModuleList()
        self.s_gat = nn.ModuleList()
        self.norm = nn.ModuleList()
        self.start_conv = nn.Conv2d(in_channels=in_dim,
                                    out_channels=residual_channels,
                                    kernel_size=(1, 1))




        if self.model_type == "GMAT":
            # Paepr eq 11: R=1+(c-1)(q^m -1)/(q -1).
            kernel_size = 7
            if dilation_exponential>1:
                self.receptive_field = int(1+(kernel_size-1)*(dilation_exponential**layers-1)/(dilation_exponential-1))
            else:
                self.receptive_field = layers*(kernel_size-1) + 1

        print("# Model Type", self.model_type)
        print("# receptive_field", self.receptive_field)
        i=0
        if dilation_exponential>1:
            rf_size_i = int(1 + i*(kernel_size-1)*(dilation_exponential**layers-1)/(dilation_exponential-1))
        else:
            rf_size_i = i*layers*(kernel_size-1)+1
        new_dilation = 1

        self.receptive_field = 10
        target_len = self.receptive_field


        self.t_len = []
        for j in range(1,layers+1):


            if self.model_type == "GMAT":
                if dilation_exponential > 1:
                    rf_size_j = int(rf_size_i + (kernel_size-1)*(dilation_exponential**j-1)/(dilation_exponential-1))
                else:
                    rf_size_j = rf_size_i+j*(kernel_size-1)

            if j % 2 == 1:
                new_dilation = 1
            elif j % 2 == 0:
                new_dilation = 2
            dilation_factor = new_dilation
            kern = 2

            in_channel = 32
            n_heads = 8
            dropout = 0
            alpha = 0.2
            self.t_gat1.append(
                GATEncoder(
                  kern= kern, dilation_factor=dilation_factor, temporal_len = target_len, n_heads=n_heads, in_channel= in_channel, num_nodes=num_nodes, mlp=[16,n_heads],mlp2=[16,32], dropout=dropout, alpha=alpha
                )
            )

            self.t_gat2.append(
                GATEncoder(
                  kern= kern, dilation_factor=dilation_factor, temporal_len = target_len, n_heads=n_heads, in_channel= in_channel, num_nodes=num_nodes, mlp=[16,n_heads],mlp2=[16,32], dropout=dropout, alpha=alpha
                )
            )

            target_len -= dilation_factor
            self.t_len.append(target_len)

            if self.model_type == "GMAT" :
                '''
                # skip_convs #
                (0): Conv2d(32, 64, kernel_size=(1, 13), stride=(1, 1))
                (1): Conv2d(32, 64, kernel_size=(1, 7), stride=(1, 1))
                (2): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
                '''
                if self.seq_length>self.receptive_field:
                    self.skip_convs.append(nn.Conv2d(in_channels=conv_channels,
                                                    out_channels=skip_channels,
                                                    kernel_size=(1, target_len)))
                else:
                    self.skip_convs.append(nn.Conv2d(in_channels=conv_channels,
                                                    out_channels=skip_channels,
                                                    kernel_size=(1, target_len)))
            dilation_factor = 1
            n_heads = 8

            self.s_gat.append(S_MutiChannel_GAT(kern, dilation_factor, n_heads, target_len, [24,16,8], [16,24,32], dropout))

            #####   GCN   ##### END

            #####   Normalization   ##### START
            if self.model_type == "GMAT":
                if self.seq_length>self.receptive_field:
                    print('1', self.seq_length - rf_size_j + 1)
                    self.norm.append(LayerNorm((residual_channels, num_nodes, target_len),elementwise_affine=layer_norm_affline))
                else:
                    print('2', self.receptive_field - rf_size_j + 1)
                    self.norm.append(LayerNorm((residual_channels, num_nodes, target_len),elementwise_affine=layer_norm_affline))
            #####   Normalization   ##### END

            new_dilation *= dilation_exponential



        self.end_conv_1 = nn.Conv2d(in_channels=skip_channels,
                                             out_channels=end_channels,
                                             kernel_size=(1,1),
                                             bias=True)
        self.end_conv_2 = nn.Conv2d(in_channels=end_channels,
                                             out_channels=out_dim,
                                             kernel_size=(1,1),
                                             bias=True)




        #####   SKIP layer   ##### START
        if self.model_type == "GMAT":
            '''
            (skip0): Conv2d(2, 64, kernel_size=(1, 19), stride=(1, 1))
            (skipE): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
            '''
            if self.seq_length > self.receptive_field:
                self.skip0 = nn.Conv2d(in_channels=in_dim, out_channels=skip_channels, kernel_size=(1, self.seq_length), bias=True)
                self.skipE = nn.Conv2d(in_channels=residual_channels, out_channels=skip_channels, kernel_size=(1, self.seq_length-self.receptive_field+1), bias=True)

            else:
                self.skip0 = nn.Conv2d(in_channels=in_dim, out_channels=skip_channels, kernel_size=(1, self.receptive_field), bias=True)
                self.skipE = nn.Conv2d(in_channels=residual_channels, out_channels=skip_channels, kernel_size=(1, 1), bias=True)
        #####   SKIP layer   ##### END

        self.idx = torch.arange(self.num_nodes).to(device)


    def forward(self, input, idx=None):
        seq_len = input.size(3)

        #**fusion**#
        #assert seq_len==self.seq_length, 'input sequence length not equal to preset sequence length'
        if hasattr(self, 'a_gmat_list'): # 代表有fusion
          #print("in pretrain!!!!!!!!!!!!")
          self.seq_length = input.shape[3] # 讓(...,4)補成(...,10)跟原本ST對齊


        # Step0: 檢查receptive_field, 不足則padding0
        if self.seq_length<self.receptive_field:
            input = nn.functional.pad(input,(self.receptive_field-self.seq_length,0,0,0))

        #print('input2', input.shape, 'self.seq_length', self.seq_length)

        # Step1: turn([64, 2, 207, 19]) to ([64, 32, 207, 19])
        x = self.start_conv(input)

        #**fusion**#
        if hasattr(self, 'a_gmat_list'): # 代表有fusion
          skip = 0

          x_all = []
          for i in range(len(adj_edges)):
              target_node = F.elu(self.test_conv_1(x[:,:,i].unsqueeze(2)))

              for out in outgoing[i]:
                  outgoing_nodes = F.elu(self.test_conv_1(x[:,:, out].unsqueeze(2)))
                  outgoing_nodes_representations = F.elu(self.test_conv_2(outgoing_nodes-target_node))
                  #print('outgoing_nodes_representations', outgoing_nodes_representations.shape)

                  ingoing_nodes = F.elu(self.test_conv_1(x[:,:, ingoing[i]]))
                  intgoing_nodes_representations = F.elu(self.test_conv_3(target_node-ingoing_nodes))
                  #print('intgoing_nodes_representations', intgoing_nodes_representations.shape)


                  outgoing_nodes_representations = ((outgoing_nodes_representations))
                  intgoing_nodes_representations = ((intgoing_nodes_representations))


                  tmp_x = torch.cat([outgoing_nodes_representations,intgoing_nodes_representations],dim=2)

                  tmp_x = self.a_gmat_list[0](tmp_x)
                  x_all.append(tmp_x[:,:,0].unsqueeze(2))
          x = torch.cat(x_all, dim=2)

        else:
          skip = self.skip0(F.dropout(input, self.dropout, training=self.training))


        #    -- START #
        # Layers : 3層 : 19->13->7->1 (取決於TCN取的維度)
        for i in range(self.layers):

            # Step2: Temporal Model --START #
            # 為上一層輸出, ex:  [64, 32, 207, 19] -> [64, 32, 207, 13] -> [64, 32, 207, 7]-> [64, 32, 207, 1]
            residual = x

            #x = x.permute(0,1,3,2)

            filter = self.t_gat1[i](x)
            filter = torch.tanh(filter)

            gate = self.t_gat2[i](x)
            gate = torch.sigmoid(gate)

            x = filter * gate
            if self.model_type == "GMAT":
                x = F.dropout(x, self.dropout, training=self.training)
            # Step2: Temporal Model --END #

            # Step3: Skip after TCN --START #
            s = x

            # fusion output:([64, 32, 207, 13])
            # skip_convsL 0:([64, 64, 207, 1])
            s = self.skip_convs[i](s)


            skip = s + skip

            # Step3: Skip after TCN --END #

            x = self.s_gat[i](x, self.predefined_A[0])

            # x 經過dilated處理後, 會減少feature維度, ex: 19->13->7->1
            # 而residual為上一層輸出, 維度為: 19, 13 ...
            # 所以需要配合x進行維度調整: [:, :, :, -x.size(3):], 然後進行elemenet-wise相加
            x = x + residual[:, :, :, -x.size(3):]

            if self.model_type == "GMAT":
                if idx is None:
                    x = self.norm[i](x,self.idx)
                else:
                    x = self.norm[i](x,idx)
            # Step4: GCN --END #

        #    -- END #

        if self.model_type == "GMAT":
            #(skipE): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
            skip = self.skipE(x) + skip

        #sys.exit()
        x = F.relu(skip)
        x = F.relu(self.end_conv_1(x))
        x = self.end_conv_2(x)

        return x

### Trainer

In [ ]:
class Trainer():
    def __init__(self, model, lrate, wdecay, clip, step_size, seq_out_len, scaler, device, cl=True):
        self.scaler = scaler

        #**fusion**#
        model.a_gmat_list = nn.ModuleList()
        in_channel = 32
        n_heads = 8
        dropout = 0
        alpha = 0.2
        t_len = -1
        model.a_gmat_list.append(
            A_GMAT_module(
              n_heads=n_heads, in_channel= in_channel, num_nodes=t_len, mlp=[n_heads],mlp2=[32], dropout=dropout, alpha=alpha
            )
        )

        model.test_conv_1 = nn.Conv2d(in_channels=32,
                                             out_channels=32,
                                             kernel_size=(1,1),
                                             bias=True)
        model.test_conv_2 = nn.Conv2d(in_channels=32,
                                             out_channels=32,
                                             kernel_size=(1,1),
                                             bias=True)
        model.test_conv_3 = nn.Conv2d(in_channels=32,
                                             out_channels=32,
                                             kernel_size=(1,1),
                                             bias=True)

        self.model = model
        self.model.to(device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lrate, weight_decay=wdecay)
        self.loss = masked_mae
        self.clip = clip
        self.step = step_size
        self.iter = 1
        self.task_level = 1
        self.seq_out_len = seq_out_len
        self.cl = cl



    def train(self, input, real_val, idx=None):
        self.model.train()
        self.optimizer.zero_grad()
        output = self.model(input, idx=idx)
        output = output.transpose(1,3)
        real = torch.unsqueeze(real_val,dim=1)

        predict = self.scaler.inverse_transform(output)

        if self.iter%self.step==0 and self.task_level<=self.seq_out_len:
            self.task_level +=1
            print("### cl learning\n iter",self.iter,"\niter%step",self.iter%self.step,"\ntask_level",self.task_level)
            print("# predict len:", len(predict[:, :, :, :self.task_level]))

        if self.cl:
            loss = masked_mae(predict[:, :, :, :self.task_level], real[:, :, :, :self.task_level], 0.0)
        else:
            loss = masked_mae(predict, real, 0.0)

        loss.backward()

        if self.clip is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.clip)

        self.optimizer.step()
        mae = masked_mae(predict,real,0.0).item()
        mape = masked_mape(predict,real,0.0).item()
        rmse = masked_rmse(predict,real,0.0).item()
        smape = masked_smape(predict,real,0.0).item()
        self.iter += 1
        return mae,mape,rmse,smape

    def eval(self, input, real_val):
        self.model.eval()
        output = self.model(input)
        output = output.transpose(1,3)
        real = torch.unsqueeze(real_val,dim=1)

        predict = self.scaler.inverse_transform(output)

        loss = self.loss(predict, real, 0.0)
        mape = masked_mape(predict,real,0.0).item()
        rmse = masked_rmse(predict,real,0.0).item()
        smape = masked_smape(predict,real,0.0).item()
        return loss.item(),mape,rmse,smape



### Parameter

In [ ]:


def str_to_bool(value):
    if isinstance(value, bool):
        return value
    if value.lower() in {'false', 'f', '0', 'no', 'n'}:
        return False
    elif value.lower() in {'true', 't', '1', 'yes', 'y'}:
        return True
    raise ValueError(f'{value} is not a valid boolean value')


parser = argparse.ArgumentParser()
parser.add_argument('--device',type=str,default='cuda',help='')
parser.add_argument('--adjtype',type=str,default='doubletransition',help='adj type')

parser.add_argument('--cl', type=str_to_bool, default=True,help='whether to do curriculum learning')

parser.add_argument('--dropout',type=float,default=0.3,help='dropout rate')

parser.add_argument('--node_dim',type=int,default=40,help='dim of nodes')
parser.add_argument('--dilation_exponential',type=int,default=1,help='dilation exponential')

parser.add_argument('--conv_channels',type=int,default=32,help='convolution channels')
parser.add_argument('--residual_channels',type=int,default=32,help='residual channels')

parser.add_argument('--in_dim',type=int,default=2,help='inputs dimension')

parser.add_argument('--batch_size',type=int,default=64,help='batch size')
parser.add_argument('--clip',type=int,default=5,help='clip')

parser.add_argument('--model_type',type=str,default='GMAT',help='model type')
parser.add_argument('--skip_channels',type=int,default=64,help='skip channels')
parser.add_argument('--end_channels',type=int,default=128,help='end channels')

parser.add_argument('--kernel_set',default=[2,3,6,7], type=int, nargs='+')


parser.add_argument('--print_every',type=int,default=50,help='')
parser.add_argument('--seed',type=int,default=101,help='random seed')
parser.add_argument('--save',type=str,default='./save/',help='save path')

parser.add_argument('--log_print', type=str_to_bool, default=False ,help='whether to load static feature')

parser.add_argument('--learning_rate',type=float,default=0.0005,help='learning rate')
parser.add_argument('--weight_decay',type=float,default=0.0001,help='weight decay rate')

parser.add_argument('--step_size1',type=int,default=300,help='step_size')
parser.add_argument('--step_size2',type=int,default=100,help='step_size')


#**fusion**#
# 把原本數值設為mobility
target = 'Mobility_Flows'
parser.add_argument('--data',type=str,default='../Data/'+target ,help='data path')
parser.add_argument('--adj_data',type=str,default='../Data/'+target+'/adj_mat_input.pkl',help='adj data path')
parser.add_argument('--num_nodes',type=int,default=84,help='number of nodes/variables')

#**fusion**#
# 用在pre-trained model 設定
target = 'GCT_Flows'
parser.add_argument('--data_pre',type=str,default='../Data/'+target ,help='data path')
parser.add_argument('--adj_data_pre',type=str,default='../Data/'+target+'/adj_mat_input.pkl',help='adj data path')
parser.add_argument('--num_nodes_pre',type=int,default=34,help='number of nodes/variables')
parser.add_argument('--expid_pre',type=int,default=202308271352,help='experiment id')

#------------------------------------------------------#

parser.add_argument('--runs',type=int,default=3,help='number of runs')
parser.add_argument('--epochs',type=int,default=180,help='')

parser.add_argument('--seq_in_len',type=int,default=8,help='input sequence length')
parser.add_argument('--seq_out_len',type=int,default=4,help='output sequence length')

parser.add_argument('--layers',type=int,default=6,help='number of layers')

parser.add_argument('--expid',type=int,default=202402230155,help='experiment id')


#args = parser.parse_args()
args=parser.parse_args(args=[])
torch.set_num_threads(3)

print('# args', args)

device = torch.device(args.device)

writer = SummaryWriter()

### Loading Mobility Flows

In [ ]:

batch_size = args.batch_size
valid_batch_size = args.batch_size
test_batch_size = args.batch_size
data = {}


for category in ['train', 'val', 'test']:

    # Loading npz
    cat_data = np.load(os.path.join(args.data, category + '.npz'))

    if args.log_print:
        print("# Loading:", category + '.npz')
        for k in cat_data.files:
            print(' - col:',k)

    data['x_' + category] = cat_data['x']     # (?, 12, 207, 2)
    data['y_' + category] = cat_data['y']     # (?, 12, 207, 2)

    if args.log_print:
        print(' - x_' +category +':', data['x_' + category].shape)
        print(' - y_' +category +':', data['y_' + category].shape)


# 使用train的mean/std來正規化valid/test #
scaler = StandardScaler(mean=data['x_train'][..., 0].mean(), std=data['x_train'][..., 0].std())

# 將欲訓練特徵改成正規化
for category in ['train', 'val', 'test']:
    data['x_' + category][..., 0] = scaler.transform(data['x_' + category][..., 0])


data['train_loader'] = DataLoaderM(data['x_train'], data['y_train'], batch_size)
data['val_loader'] = DataLoaderM(data['x_val'], data['y_val'], valid_batch_size)
data['test_loader'] = DataLoaderM(data['x_test'], data['y_test'], test_batch_size)
data['scaler'] = scaler

sensor_ids, sensor_id_to_ind, adj_mx = load_adj(args.adj_data,args.adjtype)   # adjtype: default='doubletransition'

adj_mx = [torch.tensor(i).to(device) for i in adj_mx]

dataloader = data.copy()


### Loading  GCT Flows

In [ ]:
#**fusion**#
batch_size = args.batch_size
valid_batch_size = args.batch_size
test_batch_size = args.batch_size
data_pre = {}

_types = ''

for category in ['train'+_types, 'val'+_types, 'test'+_types]:

    print("# Loading:", category + '.npz')

    # Loading npz
    cat_data = np.load(os.path.join(args.data_pre, category + '.npz'))

    data_pre['x_' + category] = cat_data['x']     # (?, 12, 207, 2)
    data_pre['y_' + category] = cat_data['y']     # (?, 12, 207, 2)

    print(cat_data['x'].shape)
    print('x[0]:',cat_data['x'][0])
    print('y[0]:',cat_data['y'][0])
    print('x[-1]',cat_data['x'][-1])
    print('y[-1]',cat_data['y'][-1])

# 使用train的mean/std來正規化valid/test #
scaler_pre = StandardScaler(mean=data_pre['x_train'+_types][..., 0].mean(), std=data_pre['x_train'+_types][..., 0].std())

# 將欲訓練特徵改成正規化
for category in ['train'+_types, 'val'+_types, 'test'+_types]:
    data_pre['x_' + category][..., 0] = scaler_pre.transform(data_pre['x_' + category][..., 0])


data_pre['train_loader'] = DataLoaderM(data_pre['x_train'+_types], data_pre['y_train'+_types], batch_size)
data_pre['val_loader'] = DataLoaderM(data_pre['x_val'+_types], data_pre['y_val'+_types], valid_batch_size)
data_pre['test_loader'] = DataLoaderM(data_pre['x_test'+_types], data_pre['y_test'+_types], test_batch_size)
data_pre['scaler'] = scaler_pre

sensor_ids_pre, sensor_id_to_ind_pre, adj_mx_pre = load_adj(args.adj_data_pre,args.adjtype)   # adjtype: default='doubletransition'

adj_mx_pre = [torch.tensor(i).to(device) for i in adj_mx_pre]

dataloader_pre = data_pre.copy()


### Loading Neighbors

In [ ]:
import pandas as pd

# load connections from csv
df = pd.read_csv(os.path.join(args.data, 'neighbors_manual_v7_rename.csv'))
connections = {}
node_ids = set()  # Set to store unique node IDs
for index, row in df.iterrows():
    node = int(row['road_segment'])
    neighbors = [int(x) for x in row[1:].dropna().tolist()]
    connections[node] = neighbors
    node_ids.add(node)
    node_ids.update(neighbors)

# Create a mapping from node IDs to indices
mapping_dict = {node_id: index for index, node_id in enumerate(sorted(node_ids))}
print(mapping_dict)


import pandas as pd
import numpy as np

# Apply the mapping to the 'road_segment' and neighbor columns
for col in df.columns:
    df[col] = df[col].map(mapping_dict)



# Get the list of IDs
ids = df['road_segment'].tolist()
#print(len(ids))
#sys.exit()

# Initialize an adjacency matrix with zeros
adj_edges = np.zeros((len(ids), len(ids)))

# Populate the adjacency matrix based on the dataframe
for index, row in df.iterrows():
    node = row['road_segment']

    for neighbor in row[1:]:
        if np.isnan(neighbor):
            continue
        adj_edges[int(node), int(neighbor)] = 1

for adj in adj_edges:
    print(adj)


#-----------------#

#a = torch.rand((64,32,44,4))

def get_outgoing_nodes(node, adj_matrix):
    """
    Get the nodes that have a direct connection from the given node.

    Parameters:
    node (int): The index of the node.
    adj_matrix (np.array): The adjacency matrix.

    Returns:
    list: A list of nodes that connect from the given node.
    """
    # Get the row corresponding to the node
    row = adj_matrix[node, :]

    # Get the indices of the elements that are 1
    outgoing_nodes = np.where(row == 1)[0]

    return outgoing_nodes.tolist()


def get_index(idx, adj_matrix):
    #target = a[:,:,2].unsqueeze(2)
    #print(target[0,0])

    # Get the nodes that connect from node 2
    outgoing_nodes = get_outgoing_nodes(idx, adj_matrix)
    '''
    outgoing_nodes_representations = a[:,:,outgoing_nodes]
    print('outgoing_nodes',outgoing_nodes)
    print('outgoing_nodes_representations',outgoing_nodes_representations[0,0])
    outgoing_mobility = outgoing_nodes_representations-target
    print('outgoing_mobility',outgoing_mobility[0,0])
    '''
    # Get the nodes that connect to node 2
    ingoing_nodes = get_outgoing_nodes(idx, adj_matrix.transpose(1,0))
    '''
    ingoing_nodes_representations = a[:,:,ingoing_nodes]
    print('ingoing_nodes',ingoing_nodes)
    print('ingoing_nodes_representations',ingoing_nodes_representations[0,0])
    ingoing_mobility = target-ingoing_nodes_representations
    print('ingoing_mobility',ingoing_mobility[0,0])
    '''
    return outgoing_nodes,ingoing_nodes

outgoing = []
ingoing = []
for idx in range(len(adj_edges)):
  out_nodes, in_nodes = get_index(idx, adj_edges)
  outgoing.append(out_nodes)
  ingoing.append(in_nodes)

print('outgoing', outgoing)
print('ingoing', ingoing)


def get_node_name(index, node_dict):
    # Create a reverse mapping from indices to node names
    index_to_node = {v: k for k, v in node_dict.items()}
    return index_to_node.get(index, "Node index not found")

#node_mapping = {'1_to_3': 0, '2_to_3': 1, '3_to_1': 2, '3_to_2': 3, '3_to_4': 4, '4_to_3': 5, '4_to_5': 6, '5_to_4': 7, '5_to_6': 8, '6_to_5': 9, '6_to_7': 10, '7_to_6': 11, '7_to_8': 12, '8_to_7': 13, '8_to_13': 14, '8_to_14': 15, '9_to_10': 16, '9_to_12': 17, '10_to_9': 18, '10_to_41': 19, '12_to_9': 20, '12_to_13': 21, '12_to_37': 22, '13_to_12': 23, '13_to_14': 24, '13_to_21': 25, '14_to_8': 26, '14_to_13': 27, '14_to_20': 28, '16_to_17': 29, '16_to_18': 30, '16_to_19': 31, '17_to_12': 32, '17_to_16': 33, '17_to_25': 34, '17_to_37': 35, '18_to_16': 36, '18_to_19': 37, '19_to_16': 38, '19_to_18': 39, '19_to_20': 40, '20_to_14': 41, '20_to_19': 42, '20_to_23': 43, '21_to_13': 44, '21_to_17': 45, '21_to_20': 46, '21_to_23': 47, '23_to_20': 48, '23_to_21': 49, '23_to_25': 50, '23_to_26': 51, '24_to_26': 52, '25_to_17': 53, '25_to_23': 54, '25_to_27': 55, '26_to_23': 56, '26_to_24': 57, '26_to_28': 58, '27_to_25': 59, '27_to_28': 60, '27_to_29': 61, '28_to_26': 62, '28_to_27': 63, '28_to_30': 64, '29_to_27': 65, '29_to_30': 66, '29_to_36': 67, '30_to_28': 68, '30_to_29': 69, '30_to_31': 70, '30_to_35': 71, '31_to_30': 72, '31_to_32': 73, '31_to_34': 74, '32_to_31': 75, '32_to_40': 76, '34_to_31': 77, '34_to_35': 78, '34_to_41': 79, '35_to_30': 80, '35_to_34': 81, '35_to_38': 82, '36_to_29': 83, '36_to_37': 84, '37_to_12': 85, '37_to_17': 86, '37_to_36': 87, '37_to_38': 88, '38_to_35': 89, '38_to_37': 90, '38_to_41': 91, '40_to_32': 92, '40_to_41': 93, '40_to_42': 94, '41_to_10': 95, '41_to_34': 96, '41_to_38': 97, '41_to_40': 98, '41_to_45': 99, '42_to_40': 100, '42_to_43': 101, '43_to_42': 102, '43_to_44': 103, '43_to_46': 104, '44_to_43': 105, '45_to_37': 106, '45_to_41': 107, '45_to_46': 108, '46_to_43': 109, '46_to_45': 110, '46_to_47': 111, '47_to_46': 112, '47_to_48': 113, '47_to_49': 114, '48_to_47': 115, '48_to_49': 116, '49_to_47': 117, '49_to_48': 118}
import pandas as pd

# Read the CSV file
df = pd.read_csv(os.path.join(args.data, 'edges_id.csv'))

# Extract the 'road_segment' column as a list
edges = df['road_segment'].tolist()

# Split the IDs by '_', convert to integers, and sort
edges_sorted = sorted(edges, key=lambda x: [int(i) for i in x.split('_to_')])

# Create a dictionary with IDs as keys and indices as values
node_mapping = {edge: i for i, edge in enumerate(edges_sorted)}

print(node_mapping, len(node_mapping.keys()))

### Trainer (pretrained model)

In [ ]:

#**fusion**#
class Trainer_pretrained():
    def __init__(self, model, lrate, wdecay, clip, step_size, seq_out_len, scaler, device, cl=True):
        self.scaler = scaler

        self.model = model
        self.model.to(device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lrate, weight_decay=wdecay)
        self.loss = masked_mae
        self.clip = clip
        self.step = step_size
        self.iter = 1
        self.task_level = 1
        self.seq_out_len = seq_out_len
        self.cl = cl


    def eval(self, input, real_val):
        #print('@Trainer_pretrained, input', input.shape)
        #print('@Trainer_pretrained, real_val', real_val.shape)
        self.model.eval()
        output = self.model(input)
        output = output.transpose(1,3)

        #**fusion**#
        output =   torch.cat([output,real_val[:,1,:output.size()[2]].unsqueeze(1)],dim=1)
        #print('@Trainer_pretrained, output2', output.shape)
        return output



### Main

In [ ]:

def main(runid):

    # if args.load_static_feature:
    #     static_feat = load_node_feature('data/sensor_graph/location.csv')
    # else:
    #     static_feat = None

    model = gginet(args.model_type,
                   args.num_nodes,
                   device,
                   predefined_A=adj_mx,

                   dropout=args.dropout,  node_dim=args.node_dim, dilation_exponential=args.dilation_exponential, conv_channels=args.conv_channels, residual_channels=args.residual_channels,
                  skip_channels=args.skip_channels, end_channels= args.end_channels,
                  seq_length=args.seq_in_len, in_dim=args.in_dim, out_dim=args.seq_out_len,
                  layers=args.layers, layer_norm_affline=True)
    engine = Trainer(model, args.learning_rate, args.weight_decay, args.clip, args.step_size1, args.seq_out_len,
                     data['scaler'], device, args.cl)


    #**fusion**#
    # 用mobility scaler試試看
    model_pre = gginet(args.model_type,

                   args.num_nodes_pre,
                   device,
                   predefined_A=adj_mx_pre,

                   dropout=args.dropout, node_dim=args.node_dim, dilation_exponential=args.dilation_exponential, conv_channels=args.conv_channels, residual_channels=args.residual_channels,
                  skip_channels=args.skip_channels, end_channels= args.end_channels,
                  seq_length=args.seq_in_len, in_dim=args.in_dim, out_dim=args.seq_out_len,
                  layers=args.layers, layer_norm_affline=True)

    engine_pre = Trainer_pretrained(
                      model_pre,
                      args.learning_rate, args.weight_decay, args.clip, args.step_size1, args.seq_out_len,
                      data['scaler'], device, args.cl)


    #**fusion**#
    # 載入model
    SAVE_PATH = args.save + "exp" + str(args.expid_pre) + "_3.pth"
    print("### loading model is:",SAVE_PATH ,'###')
    checkpoint = torch.load(SAVE_PATH)
    engine_pre.model.load_state_dict(checkpoint['model_state_dict'])
    engine_pre.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    loss = checkpoint['loss']

    print("start training...",flush=True)
    his_loss =[]
    val_time = []
    train_time = []
    minl = 1e5
    start_epoch=0
    SAVE_PATH = ""
    train_loss_epoch = []  # 紀錄train在epoch收斂
    valid_loss_epoch = []  # 紀錄valid在epoch收斂

    for i in range(start_epoch,start_epoch+args.epochs+1):

        train_loss = []
        train_mape = []
        train_rmse = []
        train_smape = []
        t1 = time.time()
        #dataloader['train_loader'].shuffle()  # 為了檢視資料先拿掉
        #**fusion**#
        permutation = np.random.permutation(dataloader['train_loader'].size)
        dataloader['train_loader'].set_permutation(permutation)
        dataloader_pre['train_loader'].set_permutation(permutation)

        #for iter, (x, y) in enumerate(dataloader['train_loader'].get_iterator()):
        #**fusion**#
        for iter, ((x, y), (x2, y2)) in enumerate(zip(dataloader['train_loader'].get_iterator(), dataloader_pre['train_loader'].get_iterator())):
            trainx = torch.Tensor(x).to(device)
            trainx= trainx.transpose(1, 3)
            trainy = torch.Tensor(y).to(device)
            trainy = trainy.transpose(1, 3)

            #**fusion**#
            trainx2 = torch.Tensor(x2).to(device)
            trainx2= trainx2.transpose(1, 3)
            trainy2 = torch.Tensor(y2).to(device)
            trainy2 = trainy2.transpose(1, 3)

            #print('x2', x2.shape)
            #**fusion**#
            # 要輸入完整trainy，因為要用到time
            pre_trained_output = engine_pre.eval(trainx2,trainy2)
            trainx = pre_trained_output

            metrics = engine.train(trainx, trainy[:,0,:,:])

            train_loss.append(metrics[0])
            train_mape.append(metrics[1])
            train_rmse.append(metrics[2])
            train_smape.append(metrics[3])

            #sys.exit()

            if iter % args.print_every == 0 :
                log = 'Iter: {:03d}, Train Loss: {:.4f}, Train MAPE: {:.4f}, Train RMSE: {:.4f}'
                print(log.format(iter, train_loss[-1], train_mape[-1], train_rmse[-1]),flush=True)
        t2 = time.time()
        train_time.append(t2-t1)
        #validation
        valid_loss = []
        valid_mape = []
        valid_rmse = []
        valid_smape = []

        s1 = time.time()
        #for iter, (x, y) in enumerate(dataloader['val_loader'].get_iterator()):
        #**fusion**#
        for iter, ((x, y), (x2, y2)) in enumerate(zip(dataloader['val_loader'].get_iterator(), dataloader_pre['val_loader'].get_iterator())):
            testx = torch.Tensor(x).to(device)
            testx = testx.transpose(1, 3)
            testy = torch.Tensor(y).to(device)
            testy = testy.transpose(1, 3)

            #**fusion**#
            testx2 = torch.Tensor(x2).to(device)
            testx2= testx2.transpose(1, 3)
            testy2 = torch.Tensor(y2).to(device)
            testy2 = testy2.transpose(1, 3)

            #**fusion**#
            # 要輸入完整trainy，因為要用到time
            pre_trained_output = engine_pre.eval(testx2,testy2)
            testx = pre_trained_output


            metrics = engine.eval(testx, testy[:,0,:,:])
            valid_loss.append(metrics[0])
            valid_mape.append(metrics[1])
            valid_rmse.append(metrics[2])
            valid_smape.append(metrics[3])
        s2 = time.time()
        log = 'Epoch: {:03d}, Inference Time: {:.4f} secs'
        print(log.format(i,(s2-s1)))
        val_time.append(s2-s1)
        mtrain_loss = np.mean(train_loss)
        mtrain_mape = np.mean(train_mape)
        mtrain_rmse = np.mean(train_rmse)
        mtrain_smape = np.mean(train_smape)

        mvalid_loss = np.mean(valid_loss)
        mvalid_mape = np.mean(valid_mape)
        mvalid_rmse = np.mean(valid_rmse)
        mvalid_smape = np.mean(valid_smape)
        #his_loss.append(mvalid_loss)
        his_loss.append(mvalid_smape)

        #writer.add_scalar("train_loss", mtrain_loss, i)
        #writer.add_scalar("valid_loss", mvalid_loss, i)

        writer.add_scalar("train_loss", mvalid_loss, i)
        writer.add_scalar("valid_loss", mvalid_loss, i)


        log = 'Epoch: {:03d}, Train Loss: {:.4f}, Train MAPE: {:.4f}, Train RMSE: {:.4f}, Valid Loss: {:.4f}, Valid MAPE: {:.4f}, Valid RMSE: {:.4f}, Training Time: {:.4f}/epoch'
        print(log.format(i, mtrain_loss, mtrain_mape, mtrain_rmse, mvalid_loss, mvalid_mape, mvalid_rmse, (t2 - t1)),flush=True)
        # 紀錄每個epoch的loss
        train_loss_epoch.append(mtrain_loss)
        valid_loss_epoch.append(mvalid_loss)

        '''
        if mvalid_loss<minl:
            torch.save(engine.model.state_dict(), args.save + "exp" + str(args.expid) + "_" + str(runid) +".pth")
            minl = mvalid_loss
        '''
        if mvalid_loss<minl:
            target_best_model = args.save + "exp" + str(args.expid) + "_" + str(runid) +".pth"
            print("### Update Best Model:",target_best_model, 'Loss:', mvalid_mape, " ###")
            #torch.save(engine.model.state_dict(), args.save + "exp" + str(args.expid) + "_" + str(runid) +".pth")
            SAVE_PATH = args.save + "exp" + str(args.expid) + "_" + str(runid) +".pth"
            torch.save({
              'epoch': i,
              'task_level': engine.task_level,
              'model_state_dict': engine.model.state_dict(),
              'optimizer_state_dict': engine.optimizer.state_dict(),
              'loss': mvalid_mape,
              'train_loss': train_loss_epoch,
              'valid_loss': valid_loss_epoch
            }, SAVE_PATH)
            minl = mvalid_loss

    print("Average Training Time: {:.4f} secs/epoch".format(np.mean(train_time)))
    print("Average Inference Time: {:.4f} secs".format(np.mean(val_time)))


    bestid = np.argmin(his_loss)

    writer.close()
    print("Training finished")
    print("The valid loss on best model is", str(round(his_loss[bestid],4)))

    #target_model = args.save + "exp" + str(args.expid) + "_" + str(runid) +".pth"
    SAVE_PATH = args.save + "exp" + str(args.expid) + "_" + str(runid) +".pth"
    print("### loading model is:",SAVE_PATH ,'###')
    #engine.model.load_state_dict(torch.load(args.save + "exp" + str(args.expid) + "_" + str(runid) +".pth"))
    checkpoint = torch.load(SAVE_PATH)
    engine.model.load_state_dict(checkpoint['model_state_dict'])
    engine.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    loss = checkpoint['loss']
    print("### Loading Model finished ###")
    print("### The valid loss on loding model is", str(round(loss,4)))

    # 只更新最後的train loss
    #checkpoint['train_loss'] = train_loss_epoch
    #checkpoint['valid_loss'] = valid_loss_epoch
    torch.save({
      'epoch': checkpoint['epoch'],  # best epoch
      'task_level': checkpoint['task_level'],
      'model_state_dict': checkpoint['model_state_dict'],
      'optimizer_state_dict': checkpoint['optimizer_state_dict'],
      'loss': checkpoint['loss'],
      'train_loss': checkpoint['train_loss'],
      'valid_loss': checkpoint['valid_loss']
    }, SAVE_PATH)
    ### 測試讀取出的model ###
    valid_loss = []
    valid_mape = []
    valid_rmse = []
    valid_smape = []
    s1 = time.time()
    #for iter, (x, y) in enumerate(dataloader['val_loader'].get_iterator()):
    #**fusion**#
    for iter, ((x, y), (x2, y2)) in enumerate(zip(dataloader['val_loader'].get_iterator(), dataloader_pre['val_loader'].get_iterator())):
        testx = torch.Tensor(x).to(device)
        testx = testx.transpose(1, 3)
        testy = torch.Tensor(y).to(device)
        testy = testy.transpose(1, 3)

        #**fusion**#
        testx2 = torch.Tensor(x2).to(device)
        testx2= testx2.transpose(1, 3)
        testy2 = torch.Tensor(y2).to(device)
        testy2 = testy2.transpose(1, 3)

        #**fusion**#
        # 要輸入完整trainy，因為要用到time
        pre_trained_output = engine_pre.eval(testx2,testy2)
        testx = pre_trained_output

        metrics = engine.eval(testx, testy[:,0,:,:])
        valid_loss.append(metrics[0])
        valid_mape.append(metrics[1])
        valid_rmse.append(metrics[2])
        valid_smape.append(metrics[3])

    mvalid_loss = np.mean(valid_loss)
    mvalid_mape = np.mean(valid_mape)
    mvalid_rmse = np.mean(valid_rmse)
    print("### 2-The valid loss on loding model is", str(round(mvalid_mape,4)))
    ### 測試讀取出的model ###

    #valid data
    outputs = []
    realy = torch.Tensor(dataloader['y_val']).to(device)
    realy = realy.transpose(1,3)[:,0,:,:]
    print('#realy', realy.shape)

    #for iter, (x, y) in enumerate(dataloader['val_loader'].get_iterator()):
    #**fusion**#
    for iter, ((x, y), (x2, y2)) in enumerate(zip(dataloader['val_loader'].get_iterator(), dataloader_pre['val_loader'].get_iterator())):
        testx = torch.Tensor(x).to(device)
        testx = testx.transpose(1,3)
        testy = torch.Tensor(y).to(device)
        testy = testy.transpose(1, 3)

        #**fusion**#
        testx2 = torch.Tensor(x2).to(device)
        testx2= testx2.transpose(1, 3)
        testy2 = torch.Tensor(y2).to(device)
        testy2 = testy2.transpose(1, 3)

        #**fusion**#
        # 要輸入完整trainy，因為要用到time
        pre_trained_output = engine_pre.eval(testx2,testy2)
        testx = pre_trained_output


        #print('testx2', testx.shape)
        with torch.no_grad():
            preds = engine.model(testx)
            preds = preds.transpose(1,3)  # 64,1,6,12

        outputs.append(preds.squeeze()) # 64,1,6,12 ->squeeze()->64,6,12

    yhat = torch.cat(outputs,dim=0)
    yhat = yhat[:realy.size(0),...]  # 5240,6,12
    print('# cat valid preds', yhat.shape)

    pred = data['scaler'].inverse_transform(yhat)

    vmae, vmape, vrmse,vsmape = metric(pred,realy)
    print("valid mape",vmape)

    #test data
    outputs = []
    realy = torch.Tensor(dataloader['y_test']).to(device)
    realy = realy.transpose(1, 3)[:, 0, :, :]

    #for iter, (x, y) in enumerate(dataloader['test_loader'].get_iterator()):
    #**fusion**#
    for iter, ((x, y), (x2, y2)) in enumerate(zip(dataloader['test_loader'].get_iterator(), dataloader_pre['test_loader'].get_iterator())):
        testx = torch.Tensor(x).to(device)
        testx = testx.transpose(1, 3)


        #**fusion**#
        testx2 = torch.Tensor(x2).to(device)
        testx2= testx2.transpose(1, 3)
        testy2 = torch.Tensor(y2).to(device)
        testy2 = testy2.transpose(1, 3)

        #**fusion**#
        # 要輸入完整trainy，因為要用到time
        pre_trained_output = engine_pre.eval(testx2,testy2)
        testx = pre_trained_output


        with torch.no_grad():
            preds = engine.model(testx)
            preds = preds.transpose(1, 3)
        outputs.append(preds.squeeze())

    yhat = torch.cat(outputs, dim=0)
    yhat = yhat[:realy.size(0), ...]  #10478, 6, 12
    print('# cat test preds', yhat.shape)

    mae = []
    mape = []
    rmse = []
    smape = []
    for i in range(args.seq_out_len):

        pred = data['scaler'].inverse_transform(yhat[:, :, i])

        real = realy[:, :, i]

        metrics = metric(pred, real)

        log = 'Evaluate best model on test data for horizon {:d}, Test MAE: {:.4f}, Test MAPE: {:.4f}, Test RMSE: {:.4f}'
        print(log.format(i + 1, metrics[0], metrics[1], metrics[2]))
        mae.append(metrics[0])
        mape.append(metrics[1])
        rmse.append(metrics[2])
        smape.append(metrics[3])

    log = '{:.2f}	{:.2f}	{:.4f}	{:.4f}	'
    print( "##### exp" + str(args.expid) + "_" + str(runid)+'	',
          log.format(mae[0], rmse[0], smape[0], mape[0]),
          log.format(mae[1], rmse[1], smape[1], mape[1]),
          log.format(mae[2], rmse[2], smape[2], mape[2]),
          log.format(mae[3], rmse[3], smape[3], mape[3]),
         )

    ### Drawing Loss Diagram ###
    fig = plt.figure(figsize=(10, 6), dpi=600)
    plt.plot(checkpoint['train_loss'], label="train loss")
    plt.plot(checkpoint['valid_loss'], label="valid loss")
    plt.legend(loc="upper right")
    plt.title('#Loss of Training', fontsize=20)
    plt.ylabel("MAPE", fontsize=14)
    plt.xlabel("Epochs", fontsize=14)
    plt.show()

    return vmae, vmape, vrmse,vsmape, mae, mape, rmse,smape

if __name__ == "__main__":

    vmae = []
    vmape = []
    vrmse = []
    vsmape = []
    mae = []
    mape = []
    rmse = []
    smape = []
    for i in range(args.runs):
        vm1, vm2, vm3,vm4, m1, m2, m3, m4 = main(i)
        vmae.append(vm1)
        vmape.append(vm2)
        vrmse.append(vm3)
        vsmape.append(vm4)
        mae.append(m1)
        mape.append(m2)
        rmse.append(m3)
        smape.append(m4)

    mae = np.array(mae)
    mape = np.array(mape)
    rmse = np.array(rmse)
    smape = np.array(smape)

    amae = np.mean(mae,0)
    amape = np.mean(mape,0)
    armse = np.mean(rmse,0)
    asmape = np.mean(smape,0)

    smae = np.std(mae,0)
    s_mape = np.std(mape,0)
    srmse = np.std(rmse,0)
    s_smape = np.std(smape,0)

    print('\n\nResults for 10 runs\n\n')
    #valid data
    print('valid\tMAE\tRMSE\tMAPE')
    log = 'mean:\t{:.4f}\t{:.4f}\t{:.4f}'
    print(log.format(np.mean(vmae),np.mean(vrmse),np.mean(vmape)))
    log = 'std:\t{:.4f}\t{:.4f}\t{:.4f}'
    print(log.format(np.std(vmae),np.std(vrmse),np.std(vmape)))
    print('\n\n')
    #test data
    print('test|horizon\tMAE-mean\tRMSE-mean\tMAPE-mean\tMAE-std\tRMSE-std\tMAPE-std')
    for i in [0,1,2,3]:
        log = '{:d}\t{:.4f}\t{:.4f}\t{:.4f}\t{:.4f}\t{:.4f}\t{:.4f}'
        print(log.format(i+1, amae[i], armse[i], amape[i], smae[i], srmse[i], s_mape[i]))
